
# Tensorflow の学習と推論を SageMaker で行う

#### ノートブックに含まれる内容

- Tensorflow を SageMaker で使うときの，基本的なやりかた
- Tensorboard の立ち上げかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: ResNet
- データ: CIFAR10
- 可視化手段: Tensorboard

## セットアップ

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

# SageMaker セッションを作成して，ロールを取得する
sagemaker_session = sagemaker.Session()
role = get_execution_role()

## データのロード

[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) データセットをローカルにダウンロードして，S3 にアップロードします．CIFAR-10 は，定番の画像データセットで，50000枚の学習用画像と，10000枚の評価用画像から構成されます．クラス数は 10 で，各画像は 32x32 ピクセルで，RGB の 3 チャネルです．

トレーニングデータとテストデータのダウンロードには 5 分程度かかります．また以下を実行する前に，**<span style="color: red;">2 行目の `data/cifar10/XX` の `XX` を指定された適切な数字に変更</span>**してください．

In [ ]:
import utils

utils.cifar10_download()
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/cifar10/XX')

## Tensorflow のスクリプトの中身を確認

SageMaker で Tensorflow の学習ジョブを走らせる際には，以下のインターフェースを備えたスクリプトを用意する必要があります．これら 4 つのメソッドは，すべて実装されていなければなりません．

### 学習時に必要なメソッド

* `model_fn`: 学習用のモデルを記述したメソッド
* `train_input_fn`: 学習用のデータに対する，前処理およびロード処理を記述したメソッド
* `eval_input_fn`: 評価用のデータに対する，前処理およびロード処理を記述したメソッド
* `serving_input_fn`: 推論の際にモデルに引き渡される特徴量を記述したメソッド

#### `model_fn` でのモデルの書きかた

`model_fn` で学習および推論に使われるモデルを定義しています．`model_fn` の中には，通常以下の 5 つの要素が含まれます．

1. [ニューラルネットワークの定義](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L96)
- [ニューラルネットワークの特徴量を適用](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L178)
- [`mode` が `predict` の場合には，ニューラルネットワークの予測結果を返す](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L186)
- [`labels` と予測値を比較して，損失関数を計算する](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L188)
- [損失関数を最適化するためのオプティマイザを作成する](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L193)
- [予測値，オプティマイザ，損失関数を返す](https://github.com/tensorflow/models/blob/master/official/mnist/mnist.py#L205)

なお，自分でアルゴリズムを記述するのではなく，既存の Tensorflow のクラスを使う場合には，`model_fn` の替わりに `estimator_fn` を記述します．また，`tf.keras` を使ってモデルを記述する場合には，同じく `model_fn` ではなく `keras_model_fn` メソッドを記述します．

### 推論時のオプショナルメソッド

また SageMaker で推論を行う際に，前処理および後処理を行う場合には，併せて以下の 2 つのメソッドを実装します．例えば推論時のリケストで pickle 形式の Body が送られてくる場合，このままでは Tensorflow の Serving が読み取ることができません（デフォルトでサポートしているリクエスト Body の形式は，CSV, JSON, Protbuf の 3 種類のみです）．そのため，Pickle を Serving が読み取れるようなデータ形式に変換する必要があります．そのときには，`input_fn` で変換処理を記述します．

特に前処理および後処理を行う必要がないときは，以下の 2 つのメソッドを記述する必要はありません．

* `input_fn`: 入力データの変形を行うメソッド
* `output_fn`: 出力データの変形を行うメソド

これらのメソッドの詳細については，[公式ドキュメント](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/tf-training-inference-code-template.html)および[SageMaker SDK の readme](https://github.com/aws/sagemaker-python-sdk#tensorflow-sagemaker-estimators) に記述がありますので，そちらを参照してください．

今回の ResNet に関しては，以下の 2 つのスクリプトがあります．メインスクリプトの方に，上で示したインターフェースメソッドが定義されています．

* [source_dir/resnet_cifar_10.py](source_dir/resnet_cifar_10.py): 学習と推論に使われるメインスクリプト
* [source_dir/resnet_model.py](source_dir/resnet_model.py): ResNet モデル

In [ ]:
!cat source_dir/resnet_cifar_10.py

## モデルの学習を実行

SageMaker SDK には，Tensorflow 専用の Estimator として，sagemaker.tensorflow.TensorFlow クラスがあります．ここでは，先ほどの `source_dir/resnet_cifar_10.py` をエントリーポイントとして指定して，Tensorflow の学習ジョブを実行します．

`fit()` を実行する際に，オプションとして `run_tensorboard_locally=True` を指定することで，デフォルトポートとして 6006 番を使用して，TensorBoasrd を起動します（当該ポートが埋まっている場合には，他のポートが使われます）．TensorBoard には，ノートブックインスタンス上からの場合 [proxy/6006/](/proxy/6006/) でアクセスします（使用するポートが異なる場合には，適宜差し替えてください）．ジョブが立ち上がるまでには数分程度かかります．

In [ ]:
from sagemaker.tensorflow import TensorFlow

source_dir = os.path.join(os.getcwd(), 'source_dir')
estimator = TensorFlow(entry_point='resnet_cifar_10.py',
                       source_dir=source_dir,
                       role=role,
                       training_steps=10,
                       evaluation_steps=3,
                       train_instance_count=1,
                       train_instance_type='ml.m4.xlarge', 
                       base_job_name='tensorboard-example')

estimator.fit(inputs, run_tensorboard_locally=True)

## モデルの推論を実行　

`deploy()` メソッドを実行することで，リアルタイムに推論結果を返すエンドポイントを立ち上げることができます．

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)